In [ ]:
!pip install pytorch-lightning transformers wandb torchmetrics deepspeed nltk

In [ ]:
!git clone https://github.com/Myashka/CQA_RLHF.git cqa

In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install cloud-tpu-client==0.10 torch==1.13.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.13-cp38-cp38-linux_x86_64.whl tensorboard-plugin-profile
    !pip install google-api-python-client==1.12.1
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
import yaml

## Run train

In [ ]:
trainer_config = dict(
    seed=42,
    model_name="EleutherAI/gpt-neo-1.3B",
    wandb=dict(
        api="60fce56bfaec85b8d6bc78bfac2086891f5afe54",
        project_name="CQA_RLHF",
        args=dict(job_type="train", group="sft", name="1.3B-tpu-kaggle"),
    ),
    data=dict(
        data_dir=r"/kaggle/input/python-qa-v2/1.0-data-div-ans-sep.json",
        max_length=512,
        batch_size=1,
        pad_for_tpu = True
    ),
    model_params=dict(
        learning_rate=2e-5,
        do_freeze=False,
        use_cache=False,
        warmup_steps=100,
        adam_betas=[0.9, 0.95],
        weight_decay=0.001,
        do_compute_metrics = False,
        do_compute_bertscore = False
    ),
    trainer=dict(
        checkpoint=dict(
            every_n_train_steps=1000,
            dirpath=r"/kaggle/working/Checkpoints",
        ),
        ckpt_path=None,
        params=dict(
            accelerator="tpu",
            max_epochs=3,
            auto_scale_batch_size=False,
            accumulate_grad_batches=1,
            gradient_clip_val=1,
            precision="16",
            val_check_interval=500,
            overfit_batches=0,  # 0 for train
            num_sanity_val_steps=2,
            log_every_n_steps = 20,
#             limit_train_batches=100,
#             strategy="deepspeed_stage_2_offload"
        ),
    ),
)

with open("trainer_config.yaml", "w") as outfile:
    yaml.dump(trainer_config, outfile, default_flow_style=False)

In [ ]:
!python /kaggle/working/CQA_RLHF/sft/train.py --config_file /kaggle/working/trainer_config.yaml

## Run test

In [ ]:
test_config = dict(
    seed=42,
    cuda=True,
    wandb=dict(
        api="text",
        args=dict(group="sft", job_type="test", name="1.3B-base-test"),
        project_name="CQA_RLHF",
        table_name="test-modelname",
    ),
    model_name="EleutherAI/gpt-neo-1.3B",
    data=dict(data_dir=r"/kaggle/input/python-qa/1.0-data-div-ans-sep.json"),
    test_model_path="checkpoint_path",
    model_params=dict(use_cache=True),
    test_params=dict(do_compute_bertscore=False, do_compute_metrics=False),
    generate_params=dict(
        do_sample=False,
        top_k=50,
        top_p=0.9,
        temperature=0,
        num_return_sequences=0,
        no_repeat_ngram_size=2,
        max_new_tokens=512,
    ),
)

with open("test_config.yaml", "w") as outfile:
    yaml.dump(test_config, outfile, default_flow_style=False)

In [ ]:
!python /content/cqa/sft/test.py --config_file /content/test_config.yaml